In [ ]:
import numpy as np
from splex import *
from pbsig import * 
from pbsig.linalg import * 
from pbsig.vis import figure_complex
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.layouts import row
output_notebook(verbose=False)

Generate a noisy circle. For any choice of fixed radius $\epsilon \in \mathbb{R}_+$, we can construct a rips filtration $\mathcal{R}(X; \epsilon)$. For example:


In [ ]:
from pbsig.vis import *
np.random.seed(1234)
theta = np.linspace(0, 2*np.pi, 8, endpoint=False)
circle = np.c_[np.sin(theta), np.cos(theta)]

eps_radius = 1.0
R = rips_filtration(circle, 1.0, p=2)

## Plot the circle + noise 
p = figure(width=300, height=300, match_aspect=True)
p.scatter(*circle.T, color="blue", size=12)

show(row(p, figure_complex(R, circle)))

The homology of the rips complex is highly dependent on the scale parameter $\epsilon$. This shows that homology, in general, is not scale-invariant. To counter this, the typical approach is to vary $\epsilon \in [0, +\infty)$ and track the changes in the homology groups. Homology groups that seem to persist for over long contiguous subsets of $[0, +\infty)$ are thought to contain highly persistent cycles, i.e. homeomorphic cycles that are stable with respect to changes in the geometry. This is a bit of a misnomer, as the actual "cycles" generating the homology groups change quite often; indeed, each inclusion map $K_{i-1} \hookrightarrow K_{i}$ induces an entirely new coset.

Persistence is often summarized with a persistence diagram.


In [ ]:
from pbsig.persistence import ph
max_radius = 3.0
K = rips_filtration(circle, max_radius, p=2)
dgms = ph(K, engine="dionysus")
dgm = dgms[1]
show(figure_dgm(dgm))

## Changing a different parameter

Indeed, suppose we scale our circle by some scaling factor $\alpha \in [0,1]$, fixing $\epsilon$ arbitrarily.


In [ ]:
from pbsig.vis import bin_color
max_scale = 2.0
alpha_family = np.linspace(0, max_scale, 100)
vine = np.c_[alpha_family*dgm["birth"], alpha_family*dgm["death"]]

p = figure_dgm()
p.scatter(vine[:,0],vine[:,1], color=bin_color(alpha_family, "turbo"))
# p.lookup(name)
show(p)


# from bokeh.layouts import gridplot
# from itertools import product
# nr, nc, r = 2, 6, 0.5 # num rows, num columns, radius 
# max_scale = 3.0
# F = lambda alpha: (alpha*circle, rips_filtration(alpha*circle, r, p=2))
# plots = [[None]*nc for r in range(nr)]
# for alpha, (i,j) in zip(np.linspace(0.0, max_scale*r, nr*nc), product(range(nr), range(nc))):
#   X, R = F(alpha)
#   plots[i][j] = figure_complex(R, X, width=100, height=100, x_range=(-max_scale*r,max_scale*r), y_range=(-max_scale*r,max_scale*r))
# show(gridplot(plots))

We have essentially the same effect, but in reverse: at first the complex is the full $n$-simplex, collapsed to trivial point in its embedding. As the complex expands, it changes from being a disk (trivial $H_1$) to a circle (non-trivial $H_1$) to $n$ disconnected points (trivial $H_1$). Essentially, rather than having an $\epsilon$-parameterized filtration over a fixed point cloud, we have an $\alpha$-parameterized family of rips filtrations $\{ \mathcal{R}_{\epsilon}(\alpha) \}_{\alpha \in A}$ of varying size but fixed filter.

<!-- Suppose one is interested in finding the interval wherein the cycle generating the single homology group $H_1$ is the most "persistent" in $\alpha$. We could build a filtration in reverse to get inclusions... -->


In [ ]:
from pbsig.vis import plot_dgm, figure_dgm
from pbsig.persistence import ph 
# X, R = F(0.50)
# dgm = ph(R, engine="dionysus")[1]
# plot_dgm(dgm)

plots = [[None]*nc for r in range(nr)]
for alpha, (i,j) in zip(np.linspace(0.0, max_scale*r, nr*nc), product(range(nr), range(nc))):
  X, R = F(alpha)
  dgm = ph(R, engine="dionysus")
  dgm = dgm[1] if 1 in dgm.keys() else np.empty((0,2),dtype=[('birth', 'f4'), ('death', 'f4')])
  # figure_complex(R, X, width=100, height=100, x_range=(-max_scale*r,max_scale*r), y_range=(-max_scale*r,max_scale*r))
  plots[i][j] = figure_dgm(dgm, width=100, height=100) # x_range=(-max_scale*r,max_scale*r), y_range=(-max_scale*r,max_scale*r)
  plots[i][j].title = None
  plots[i][j].xaxis.axis_label = None
  plots[i][j].yaxis.axis_label = None
  # plots[i][j].xaxis.visible = False
show(gridplot(plots))

Observe we have a few "phase changes" that occur here. Initially, there is no $H_1$ class. As the the complex expands beyond the trivial point, a point appears representing the aforementioned cycle. Since the persistence diagram is stable, we know this point must appear first on the diagonal---it then travels further from the diagonal until the cycle is broken apart. In this situation, it becomes an *essential class*; we have not expanded the rips complex to a large enough $\epsilon$ to see triangles close up the cycle. The birth of the cycle grows infinitely as $\alpha \to \infty$.

Suppose we wanted to determine the choices of $\alpha$ where this cycle existed. We can peek at these via a set of multiplicity queries:


In [ ]:
from pbsig.betti import mu_query
R = (0.5, 1.0, 1.5, 2.0)
S = simplicial_complex(faces(SimplexTree([np.arange(8)]), 2))
Q = [mu_query(S, R=R, f=flag_weight(alpha*circle), p=1, normed=False) for alpha in alpha_family]

mult_H1 = [mu() for mu in Q]
p = figure(
  width=350, height=200, 
  title=f"Circle multiplicities for R={R}", x_axis_label="alpha (scaling factor)", y_axis_label="multiplicity"
)
p.step(alpha_family, np.array(mult_H1, dtype=int))
p.yaxis.minor_tick_line_alpha = 0
show(p)

Equivalently, we can amortize the cost of creating so many matrices by re-using the results from previous computations.


In [ ]:
from pbsig.betti import MuFamily
F = [flag_weight(alpha*circle) for alpha in alpha_family]
mu_f = MuFamily(S, family=F, p=1, form="array")
mu_f.precompute(R=R, progress=True)

p = figure(
  width=350, height=200, 
  title=f"Circle multiplicities for R={R}", x_axis_label="alpha (scaling factor)", y_axis_label="multiplicity"
)
p.step(alpha_thresholds, np.array(mu_f(smoothing=None), dtype=int))
p.yaxis.minor_tick_line_alpha = 0
show(p)

Let's look at the constitutive terms that make up this multiplicity queries


In [ ]:
p = figure(
  width=350, height=200, 
  title=f"Circle multiplicities for R={R}", 
  x_axis_label="alpha (scaling factor)", 
  y_axis_label="Constititive ranks"
)
mu_terms = mu_f(smoothing=None, terms=True).T
p.step(alpha_thresholds, mu_terms[:,0], line_color="red")
p.step(alpha_thresholds, -mu_terms[:,1], line_color="green")
p.step(alpha_thresholds, -mu_terms[:,2], line_color="blue")
p.step(alpha_thresholds, mu_terms[:,3], line_color="orange")
p.step(alpha_thresholds, mu_f(smoothing=None), line_color="black")
p.yaxis.minor_tick_line_alpha = 0
show(p)

Now, let's look at a continuous relaxation of both the multiplicity function and its constitituive terms.


In [ ]:
## constituive terms
figures = [[] for i in range(4)]
for w, normed in [(0.0,False), (0.0,True), (0.30,False), (0.30,True)]:
  mu_f.precompute(R=R, w=w, normed=normed, progress=True)
  mu_terms = mu_f(smoothing=None, terms=True).T
  mu_terms_nuclear = mu_f(smoothing=False, terms=True).T
  mu_terms_sgn_approx = mu_f(smoothing=sgn_approx(eps=1e-1, p=1.5), terms=True).T
  for i in range(4):
    p = figure(
      width=200, height=200, 
      title=f"w:{w}, normed:{normed}", x_axis_label="alpha (scaling factor)", y_axis_label="multiplicity"
    )
    p.yaxis.minor_tick_line_alpha = 0
    p.line(alpha_thresholds, mu_terms_nuclear[:,i], line_color = "orange", line_width=2.0)
    p.line(alpha_thresholds, mu_terms_sgn_approx[:,i], line_color = "blue",  line_width=1.5)
    p.step(alpha_thresholds, mu_terms[:,i], line_color = "black", line_width=1.0)
    figures[i].append(p)

show(row([column(f) for f in figures]))

The weighted combinatorial laplacian is unstable whenever $w > 0$, due to the fact that $1/\epsilon$ can produce arbitrarily large values as $\epsilon \to 0^+$. This is not a problem for the normalized laplacian, as the spectrum is always bounded in the range $[0, p+2]$.

**TAKEAWAY**: If $w > 0$, you need degree normalization to stabilize the spectrum. Though the nuclear norm is at times quite similar to the rank, it can also differ greatly.

------------------------------------------------------------------------

From now on, let's only consider the normalized weighted combinatorial laplacian. Let's look at the effect of w


In [ ]:
W = [0.0, 0.20, 0.40, 0.60, 0.80]
fig_kwargs = dict(width=200, height=150)

## Try varying epsilon in [1e-1, 100] to see interpolation of behavior
figures = [[] for i in range(len(W))]
for i,w in enumerate(W):
  mu_f.precompute(R=R, w=w, normed=True, progress=True)
  for j,ef in enumerate([None, sgn_approx(eps=1e-1, p=1.2), False]):
    mu_terms = mu_f(smoothing=ef, terms=True).T
    p = figure(**(fig_kwargs | dict(title=f"w:{w}, smooth:{j}")))
    p.line(alpha_thresholds, mu_terms[:,0], line_color="red")
    p.line(alpha_thresholds, -mu_terms[:,1], line_color="green")
    p.line(alpha_thresholds, -mu_terms[:,2], line_color="blue")
    p.line(alpha_thresholds, mu_terms[:,3], line_color="orange")
    p.line(alpha_thresholds, mu_terms.sum(axis=1), line_color="black")
    p.yaxis.minor_tick_line_alpha = 0
    figures[i].append(p)

show(column([row(f) for f in figures]))

The nuclear norm not only causes a spurious modes, it effectively removes the existing maximizer. Changing `w` has the effect of smoothing the objective and expanding the feasible set when appropriately chosen and the sgn approximation is used, whereas seemingly has neglible on the nuclear norm.

Let's evaluate the Moreau envelope on the nuclear norm.


In [ ]:
w = 10000.35
mu_f.precompute(R=R, w=w, normed=True, progress=True) 

In [ ]:
from pbsig.linalg import *
from pbsig.utility import *


mu_terms = mu_f(smoothing=True, terms=True).T
mu_mats = [mu_query_mat(S=S, R=np.append(R, w), f=f, p=1, form = 'array', normed=True) 
           for f in mu_f.family]
me = lambda M, t: prox_nuclear(M, t)[1]

t = 0.50
mu_moreau = [[] for i in range(4)]
for M1, M2, M3, M4 in progressbar(mu_mats):
  mu_moreau[0].append(me(M1, t))
  mu_moreau[1].append(-me(M2, t))
  mu_moreau[2].append(-me(M3, t))
  mu_moreau[3].append(me(M4, t))
mu_moreau = np.array(mu_moreau).T

magnitudes = abs(mu_terms).sum(axis=1)
magnitudes = np.isclose(magnitudes, 0.0).astype(int)
feasible = np.flatnonzero(np.diff(magnitudes) != 0.0)

p = figure(**(fig_kwargs | dict(title=f"w:{w}, smooth:{j}")))
p.line(alpha_thresholds, mu_terms.sum(axis=1), line_color="black")
p.line(alpha_thresholds, mu_moreau.sum(axis=1), line_color="black", line_dash="dotted", line_width=1.5)
p.yaxis.minor_tick_line_alpha = 0
show(p)

In [ ]:
import line_profiler

profile = line_profiler.LineProfiler()
profile.add_function(mu_f.precompute)
profile.enable_by_count()
mu_f.precompute(R=R, w=w, normed=True, progress=True) 
profile.print_stats(output_unit=1e-3)

In [ ]:
def moreau_loss(x: ArrayLike, eps: float, p: float = 1.0, t: float = 1.0):
  """
  Parameters:
    x: ndarray, spmatrix, or LinearOperator
    t: proximal scaling operator 
  """
  from scipy.sparse import spmatrix
  from scipy.optimize import minimize
  if isinstance(x, np.ndarray) and all(np.ravel(x == 0)):
    return 0
  elif isinstance(x, spmatrix) and len(x.data) == 0:
    return 0
  sf = sgn_approx(eps=eps, p=p)  
  x_ew = np.linalg.eigvalsh(x)
  def sgn_approx_cost(ew_hat: ArrayLike, t: float):
    return sum(sf(ew_hat)) + (1/(t*2)) * np.linalg.norm(sf(ew_hat) - sf(x_ew))**2
  w = minimize(sgn_approx_cost, x0=x_ew, args=(t), tol=1e-15, method="Powell")
  if w.status != 0:
    import warnings
    warnings.warn("Did not converge to sign vector prox")
  ew = w.x if w.status == 0 else x_ew
  return sgn_approx_cost(ew, t)


t, eps = 0.80, 1e-1
mu_moreau = [[] for i in range(4)]
for M1, M2, M3, M4 in mu_mats:
  mu_moreau[0].append(moreau_loss(M1.todense(), eps=eps, p=1.5, t=t))
  mu_moreau[1].append(-moreau_loss(M2.todense(), eps=eps, p=1.5, t=t))
  mu_moreau[2].append(-moreau_loss(M3.todense(), eps=eps, p=1.5, t=t))
  mu_moreau[3].append(moreau_loss(M4.todense(), eps=eps, p=1.5, t=t))
mu_moreau = np.array(mu_moreau).T

p = figure(**(fig_kwargs | dict(title=f"w:{w}, t:{t}, eps:{eps}")))
p.line(alpha_thresholds, mu_terms[:,0], line_color="red")
p.line(alpha_thresholds, mu_moreau[:,0], line_color="red", line_dash="dotted", line_width=1.5)
p.line(alpha_thresholds, -mu_terms[:,1], line_color="green")
p.line(alpha_thresholds, -mu_moreau[:,1], line_color="green", line_dash="dotted", line_width=1.5)
p.line(alpha_thresholds, -mu_terms[:,2], line_color="blue")
p.line(alpha_thresholds, -mu_moreau[:,2], line_color="blue", line_dash="dotted", line_width=1.5)
p.line(alpha_thresholds, mu_terms[:,3], line_color="orange")
p.line(alpha_thresholds, mu_moreau[:,3], line_color="orange", line_dash="dotted", line_width=1.5)
q = figure(**(fig_kwargs | dict(title=f"w:{w}, t:{t}, eps:{eps}")))
q.line(alpha_thresholds, mu_terms.sum(axis=1), line_color="black")
p.line(alpha_thresholds, mu_terms.sum(axis=1), line_color="black")
q.line(alpha_thresholds, mu_moreau.sum(axis=1), line_color="black", line_dash="dotted", line_width=1.5)
p.line(alpha_thresholds, mu_moreau.sum(axis=1), line_color="black", line_dash="dotted", line_width=1.5)
q.yaxis.minor_tick_line_alpha = 0
show(row(p, q))

In [ ]:
from pbsig.linalg import *
w = 0.35
mu_f.precompute(R=R, w=w, normed=True, progress=True) 
mu_terms = mu_f(smoothing=True, terms=True).T
mu_mats = [mu_query_mat(S=S, R=R, f=f, p=1, form = 'array', normed=True, w=w) for f in mu_f.family]
me = lambda M, t: prox_nuclear(M, t)[1]

t = 5.0
mu_moreau = [[] for i in range(4)]
for M1, M2, M3, M4 in mu_mats:
  mu_moreau[0].append(me(M1, t))
  mu_moreau[1].append(-me(M2, t))
  mu_moreau[2].append(-me(M3, t))
  mu_moreau[3].append(me(M4, t))
mu_moreau = np.array(mu_moreau).T

p = figure(**(fig_kwargs | dict(title=f"w:{w}, smooth:{j}")))
p.line(alpha_thresholds, mu_terms[:,0], line_color="red")
p.line(alpha_thresholds, mu_moreau[:,0], line_color="red", line_dash="dotted", line_width=1.5)
p.line(alpha_thresholds, -mu_terms[:,1], line_color="green")
p.line(alpha_thresholds, -mu_moreau[:,1], line_color="green", line_dash="dotted", line_width=1.5)
p.line(alpha_thresholds, -mu_terms[:,2], line_color="blue")
p.line(alpha_thresholds, -mu_moreau[:,2], line_color="blue", line_dash="dotted", line_width=1.5)
p.line(alpha_thresholds, mu_terms[:,3], line_color="orange")
p.line(alpha_thresholds, mu_moreau[:,3], line_color="orange", line_dash="dotted", line_width=1.5)
p.line(alpha_thresholds, mu_terms.sum(axis=1), line_color="black")
p.line(alpha_thresholds, mu_moreau.sum(axis=1), line_color="black", line_dash="dotted", line_width=1.5)
p.yaxis.minor_tick_line_alpha = 0
show(p)

In [ ]:
## Testing the moreau envelope 
from pbsig.linalg import prox_nuclear
X = np.random.uniform(size=(10,10))
X = X @ X.T

from pbsig.linalg import soft_threshold
t = 0.15
ew, ev = np.linalg.eigh(X)
assert np.isclose(sum(np.linalg.eigvalsh(ev @ np.diag(ew) @ ev.T)), sum(ew))
assert np.isclose(sum(np.linalg.eigvalsh(ev @ np.diag(soft_threshold(ew, 0.15)) @ ev.T)), sum(soft_threshold(ew, 0.15)))
P = ev @ np.diag(soft_threshold(ew, 0.15)) @ ev.T  #  proximal operator 
assert np.isclose(sum(np.linalg.eigvalsh(P)), sum(soft_threshold(ew, 0.15)))
me = sum(soft_threshold(ew, 0.15)) + (1/(2*0.15))*np.linalg.norm(X - P, 'fro')**2
P, mf, _ = prox_nuclear(X, t=t) # 35.36999221118293
assert np.isclose(me, mf)


A = np.random.uniform(size=(10,10))
A = A @ A.T
print(np.linalg.norm(A - X, 'fro')**2)
ew_x, U =np.linalg.eigh(X)
ew_v, V =np.linalg.eigh(A)
S = np.diag(ew_x)
D = np.diag(ew_v)

np.trace(S**2) + np.trace(D**2) - 2*np.trace(V.T @ U @ S @ U.T @ V @ D)
np.trace(S**2) + np.trace(D**2) - 2*np.trace(S**2 @ U.T @ X @ D @ X.T @ U)

sum((np.diag(A) - np.diag(X))**2)
sum((np.diag(A)**2 - np.diag(X)**2))



from pbsig.linalg import moreau
sum(moreau(ew, t))

Investigate why the nuclear and moreau are so far away from the rank representations (and why they are biased)? This does not have anything to do with *w* apparently, as *w = 0* yields the same behavior. So let's just conclude the nuclear is not a good subtitute for the rank function. Instead, let's develop a proximal operator for the sign approximation function.


In [ ]:
from pbsig.linalg import eigh_solver, eigvalsh_solver
from pbsig.betti import mu_query_mat
sf = sgn_approx(eps=1e-2, p=1.2)  
LM = mu_query_mat(S, R=R, f=F[jj], p=1, form="array")
Y = LM[3].todense()
#ew, ev = eigh_solver(Y)(Y)

y_shape = Y.shape
y = np.ravel(Y)
def moreau_cost(y_hat: ArrayLike, t: float = 1.0):
  Y_hat = y_hat.reshape(y_shape)
  ew = np.maximum(np.real(np.linalg.eigvals(Y_hat)), 0.0) # eiegnvalues can be negative, so we project onto the PSD cone!
  ew_yhat = sum(sf(ew)) 
  if t == 0.0: 
    return ew_yhat
  return ew_yhat + (1/(t*2))*np.linalg.norm(Y_hat - Y, "fro")**2
# ev @ np.diag(sf(ew)) @ ev.T

from scipy.optimize import minimize
y_noise = y+np.random.uniform(size=len(y), low=0, high=0.01)
w = minimize(moreau_cost, x0=y_noise, args=(0.01))
Z = w.x.reshape(y_shape)
print(f"Status: {w.status}, total error: {np.linalg.norm(Z - Y, 'fro')}, Num it: {w.nit}, Num evals: {w.nfev} \nMessage: {w.message}")

## Try a vector based optimization
from scipy.optimize import minimize
y_ew = np.linalg.eigvalsh(Y)
def sgn_approx_cost(ew_hat: ArrayLike, t: float = 1.0):
  return sum(sf(ew_hat)) + (1/(t*2)) * np.linalg.norm(sf(ew_hat) - sf(y_ew))**2
y_ew_noise = y_ew + np.random.uniform(size=len(y_ew), low=0.0, high=0.50)
w = minimize(sgn_approx_cost, x0=y_ew_noise, args=(0.5), tol=1e-15, method="Powell")
print(f"Status: {w.status}, total error: {np.linalg.norm(y_ew - w.x)}, Num it: {w.nit}, Num evals: {w.nfev} \nMessage: {w.message}")



eigvalsh_solver(Z)(Z)

mu_query(S, R, f)
solver = eigh_solver(x)
ew, ev = solver(x)


x0 

# j = np.searchsorted(alpha_thresholds, 0.90)
# mu = mu_query(S, R=np.append(R, 0.35), f=F[j], p=1, normed=True)
# mu(smoothing=None, terms=False) # 1 
# mu(smoothing=None, terms=True) # 8,  -8, -16,  17
# mu(smoothing=sgn_approx(eps=1e-2, p=1.2), terms=False) # 1.0004954387928944
# mu(smoothing=sgn_approx(eps=0.90, p=1.0), terms=False) # 0.5891569860030863
# mu(smoothing=False, terms=True) #  8., -16., -16.,  24.

# jj = np.searchsorted(alpha_thresholds, 1.15)
# f=F[jj]
# spectral_rank(EW[0])

# mu = mu_query(S, R=np.append(R, 0.35), f=F[jj], p=1, normed=True)
# mu(smoothing=False, terms=True)

In [ ]:
mu_f.precompute(R=R, normed=False, progress=True)

Use discrete vineyards to get an idea of what the

# st = SimplexTree(complete_graph(X.shape\[0\]))

# st.expand(2)

# S = st

N, M = 20, 24 SW = sliding_window(sw_f, bounds=(0, 12*np.pi)) d, tau = sw_parameters(bounds=(0,12*np.pi), d=M, L=6) #S = delaunay_complex(F(n=N, d=M, tau=tau)) X = SW(n=N, d=M, tau=tau) \# r = enclosing_radius(X)\*0.60 \# S = rips_complex(X, r, 2) show(plot_complex(S, X\[:,:2\]))

## Plot

scatters = \[\] for t in np.linspace(0.50*tau, 1.50*tau, 10): X_delay = SW(n=N, d=M, tau=t) p = figure(width=150, height=150, toolbar_location=None) p.scatter(*pca(X_delay).T) scatters.append(plot_complex(S, pos=pca(X_delay), width=125, height=125)) show(row(*scatters))

from pbsig.persistence import ph from pbsig.vis import plot_dgm K = filtration(S, f=flag_weight(X)) dgm = ph(K, engine="dionysus") plot_dgm(dgm\[1\])

from pbsig.betti import MuSignature, mu_query from pbsig.linalg import \* R = np.array(\[4, 4.5, 6.5, 7.5\]) T_dom = np.append(np.linspace(0.87*tau, tau, 150, endpoint=False), np.linspace(tau, tau*1.12, 150, endpoint=False)) t_family = \[flag_weight(SW(n=N, d=M, tau=t)) for t in T_dom\]

MU_f = mu_query(S, R=R, f=flag_weight(SW(n=N, d=M, tau=tau)), p=1, form="array")

Generate a noisy circle


In [ ]:
np.random.seed(1234)
theta = np.linspace(0, 2*np.pi, 80, endpoint=False)
circle = np.c_[np.sin(theta), np.cos(theta)]
noise_scale = np.random.uniform(size=circle.shape[0], low=0.90, high=1.10)
noise_scale = np.c_[noise_scale, noise_scale]
noise = np.random.uniform(size=(10, 2), low=-1, high=1)
X = np.vstack((circle*noise_scale, noise))

## Plot the circle + noise 
p = figure(width=400, height=200)
p.scatter(X[:,0], X[:,1], color="blue")
p.scatter(*noise.T, color="red")
show(p)

````{=html}
<!-- 

alpha_thresholds = np.linspace(1e-12, max_scale*r, 100)
vine = np.vstack([ph(F(alpha)[1], engine="dionysus")[1] for alpha in alpha_thresholds])

from bokeh.models import Range1d
p = figure_dgm(vine[-1,:])
p.scatter(np.ravel(vine['birth']), np.ravel(vine['death']))
p.x_range = Range1d(0, max_scale*r)
p.y_range = Range1d(0, max_scale*r)
show(p)
``` -->
````